In [1]:
!pip install torchaudio
!pip install transformers
!pip install datasets
!pip install lang_trans
!pip install arabic_reshaper
!pip install python-bidi
!pip install pydub
!pip install soundfile
!pip install jiwer
!pip install PyArabic

In [5]:
import os
import re
import sys
import json
import torch
import jiwer
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import transformers
import pandas as pd
from torch import nn
import seaborn as sns
import torch.nn as nn
import soundfile as sf
import arabic_reshaper
from pyarabic import araby
from packaging import version
from pydub import AudioSegment
import matplotlib.pyplot as plt
from trainer import CTCTrainer
from pydub.utils import mediainfo
from argparse import ArgumentParser
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from models import Wav2Vec2ClassificationModel
from processors import CustomWav2Vec2Processor
from typing import Any, Dict, List, Optional, Union
from transformers import HfArgumentParser,TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import is_apex_available,set_seed ,Trainer,Wav2Vec2FeatureExtractor
from arg_parsers import DataTrainingArguments, ModelArguments, DataCollatorCTCWithPadding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel
% matplotlib inline

UsageError: Line magic function `%` not found.


In [4]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/All_poems.zip .

In [ ]:
if os.path.exists('dataset'):
  if len(os.listdir('dataset')) == 0:
    os.system('unzip All_poems.zip -d dataset')
else:
  os.system('unzip All_poems.zip -d dataset')

In [ ]:
!mkdir -p dataset_wav

In [ ]:
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/testset.csv .
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/trainset.csv .
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/valset.csv .

In [ ]:
metadata_test_path = 'testset.csv'
metadata_train_path = 'trainset.csv'
metadata_val_path = 'valset.csv'
dataset_folder = 'dataset'
dataset_wav_folder = 'dataset_wav'

In [ ]:
train_metadata = pd.read_csv(metadata_train_path)
test_metadata = pd.read_csv(metadata_test_path)
val_metadata = pd.read_csv(metadata_val_path)

In [ ]:
sample_rates = set()
for file_path in itertools.chain(
    train_metadata['Utterance name'],
    test_metadata['Utterance name'],
    val_metadata['Utterance name']
  ):
  complete_path = f'{dataset_folder}/{file_path}'
  complete_wav_path = f'{dataset_wav_folder}/{file_path}'
  # os.system(f'ffmpeg -i {complete_path} {complete_wav_path}')
  audio = AudioSegment.from_file(complete_path)
  sample_rates.add(audio.frame_rate)
  audio.export(f'{dataset_wav_folder}/{file_path}', format='wav')
sample_rates

{44100, 48000}

In [ ]:
# the following can be used
# https://beta.quod.ai/github/huggingface/transformers?question_modal=true&question_public_id=8757&from_search=true

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
if is_apex_available():
    from apex import amp

In [ ]:
if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast

logger = logging.getLogger(__name__)

## getting training arguments

In [ ]:
parser = HfArgumentParser(
        (ModelArguments, DataTrainingArguments, TrainingArguments)
    )
if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
    # If we pass only one argument to the script and it's the path to a json file,
    # let's parse it to get our arguments.
    model_args, data_args, training_args = parser.parse_json_file(
        json_file=os.path.abspath(sys.argv[1])
    )
else:
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()

## logging setup

In [ ]:
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
logger.setLevel(
    logging.INFO if is_main_process(training_args.local_rank) else logging.WARN
)

# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
# Set the verbosity to info of the Transformers logger (on main process only):
if is_main_process(training_args.local_rank):
    transformers.utils.logging.set_verbosity_info()

# Set seed before initializing model.
set_seed(training_args.seed)


## check for previous checkpoints

In [ ]:
# Detecting last checkpoint.
last_checkpoint = None
if (
    os.path.isdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

## dataset processing

In [ ]:
train_metadata = pd.read_csv(metadata_train_path)
val_metadata = pd.read_csv(metadata_val_path)

train_dataset = Dataset.from_pandas(train_metadata)
eval_dataset = Dataset.from_pandas(val_metadata)
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16_000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True,
)
processor = CustomWav2Vec2Processor(feature_extractor=feature_extractor)
model = Wav2Vec2ClassificationModel.from_pretrained(
    "bakrianoo/sinai-voice-ar-stt",
    attention_dropout=0.01,
    hidden_dropout=0.01,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.01,
    gradient_checkpointing=True,
    num_attention_heads=4,
)

if model_args.freeze_feature_extractor:
    model.freeze_feature_extractor()

if data_args.max_train_samples is not None:
    train_dataset = train_dataset.select(range(data_args.max_train_samples))

if data_args.max_val_samples is not None:
    eval_dataset = eval_dataset.select(range(data_args.max_val_samples))
# Preprocessing the datasets.
# We need to read the aduio files as arrays and tokenize the targets.
resamplers = {  # The dataset contains all the uncommented sample rates
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    # 32000: torchaudio.transforms.Resample(32000, 16000),
}

labels = {
    bahr: bahr_index
    for bahr_index, bahr in enumerate(sorted(set(train_metadata["Bahr"])))
}
print("labels are:", labels)
print("len:", len(labels))

def speech_file_to_array_fn(batch):
    start = 0
    stop = 20
    srate = 16_000
    speech_array, sampling_rate = torchaudio.load(
        f'../dataset_wav/{batch["Utterance name"]}'
    )
    speech_array = speech_array[0]
    batch["speech"] = resamplers[sampling_rate](speech_array).squeeze().numpy()
    batch["sampling_rate"] = srate
    batch["parent"] = labels[batch["Bahr"]]
    return batch

train_dataset = train_dataset.map(
    speech_file_to_array_fn,
    remove_columns=train_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)
eval_dataset = eval_dataset.map(
    speech_file_to_array_fn,
    remove_columns=eval_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)

def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
    batch["input_values"] = processor(
        batch["speech"], sampling_rate=batch["sampling_rate"][0]
    ).input_values
    batch["labels"] = batch["parent"]
    return batch

train_dataset = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)
eval_dataset = eval_dataset.map(
    prepare_dataset,
    remove_columns=eval_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)

## training

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids.argmax(-1)
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds)
    matrix = confusion_matrix(labels, preds)
    print(matrix)
    return {"accuracy": acc}
# Data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
# Initialize our Trainer
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=processor.feature_extractor,
)
# Training
if training_args.do_train:
    if last_checkpoint is not None:
        checkpoint = last_checkpoint
    elif os.path.isdir(model_args.model_name_or_path):
        checkpoint = model_args.model_name_or_path
    else:
        checkpoint = None
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    trainer.save_model()

    # save the feature_extractor and the tokenizer
    if is_main_process(training_args.local_rank):
        processor.save_pretrained(training_args.output_dir)

    metrics = train_result.metrics
    max_train_samples = (
        data_args.max_train_samples
        if data_args.max_train_samples is not None
        else len(train_dataset)
    )
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

# Evaluation
results = {}
if training_args.do_eval:
    logger.info("*** Evaluate ***")
    metrics = trainer.evaluate()
    max_val_samples = (
        data_args.max_val_samples
        if data_args.max_val_samples is not None
        else len(eval_dataset)
    )
    metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)